In [2]:
from pymongo import MongoClient, InsertOne
from prettytable import PrettyTable

# Establish client connection
client = MongoClient('mongodb://localhost:27017/')
db = client['tidar_db']
courses_collection = db['courses']

# Bulk insert of courses with student enrollments
operations = [
    InsertOne({'course': 'Mechanics 101', 'enrollments': 40, 'department': 'Physics'}),
    InsertOne({'course': 'Quantum Mechanics', 'enrollments': 35, 'department': 'Applied Physics'}),
    InsertOne({'course': 'Thermodynamics', 'enrollments': 30, 'department': 'Engineering Physics'}),
    InsertOne({'course': 'Optics', 'enrollments': 25, 'department': 'Physics and Astronomy'}),
    InsertOne({'course': 'Data Structures', 'enrollments': 60, 'department': 'Computer Science'}),
    InsertOne({'course': 'Machine Learning', 'enrollments': 50, 'department': 'Computer Engineering'}),
    InsertOne({'course': 'Calculus I', 'enrollments': 55, 'department': 'Mathematics'}),
    InsertOne({'course': 'Linear Algebra', 'enrollments': 45, 'department': 'Mathematics'}),
    InsertOne({'course': 'Introduction to Robotics', 'enrollments': 38, 'department': 'Robotics Engineering'}),
    InsertOne({'course': 'Signal Processing', 'enrollments': 28, 'department': 'Electrical and Physics Engineering'})
]
courses_collection.bulk_write(operations)
print("Courses inserted successfully!\nData:")

# Collect all data
courses_data = courses_collection.find()

# Show table
table = PrettyTable()
table.field_names = ["Course Name", "Department", "Enrollments"]

for course in courses_data:
    table.add_row([course['course'], course['department'], course['enrollments']])

print(table)

Courses inserted successfully!
Data:
+--------------------------+------------------------------------+-------------+
|       Course Name        |             Department             | Enrollments |
+--------------------------+------------------------------------+-------------+
|      Mechanics 101       |              Physics               |      40     |
|    Quantum Mechanics     |          Applied Physics           |      35     |
|      Thermodynamics      |        Engineering Physics         |      30     |
|          Optics          |       Physics and Astronomy        |      25     |
|     Data Structures      |          Computer Science          |      60     |
|     Machine Learning     |        Computer Engineering        |      50     |
|        Calculus I        |            Mathematics             |      55     |
|      Linear Algebra      |            Mathematics             |      45     |
| Introduction to Robotics |        Robotics Engineering        |      38     |
|  

In [3]:
# Query courses ONLY in Computer Science or Mathematics Department
query = {'department': {'$in': ['Computer Science', 'Mathematics']}}

# Collect data from query above
courses_data = courses_collection.find(query)

# Show data with table
table = PrettyTable()
table.field_names = ["Course Name", "Department", "Enrollments"]

for course in courses_data:
    table.add_row([course['course'], course['department'], course['enrollments']])

print(table)

+-----------------+------------------+-------------+
|   Course Name   |    Department    | Enrollments |
+-----------------+------------------+-------------+
| Data Structures | Computer Science |      60     |
|    Calculus I   |   Mathematics    |      55     |
|  Linear Algebra |   Mathematics    |      45     |
| Data Structures | Computer Science |      60     |
|    Calculus I   |   Mathematics    |      55     |
|  Linear Algebra |   Mathematics    |      45     |
+-----------------+------------------+-------------+


In [4]:
# Query courses in Computer
query = {'department': {'$regex': 'Computer', '$options': 'i'}}

# Collect data from query above
courses_data = courses_collection.find(query)

# Show data with table
table = PrettyTable()
table.field_names = ["Course Name", "Department", "Enrollments"]

for c in courses_data:
    table.add_row([c['course'], c['department'], c['enrollments']])

print(table)

+------------------+----------------------+-------------+
|   Course Name    |      Department      | Enrollments |
+------------------+----------------------+-------------+
| Data Structures  |   Computer Science   |      60     |
| Machine Learning | Computer Engineering |      50     |
| Data Structures  |   Computer Science   |      60     |
| Machine Learning | Computer Engineering |      50     |
+------------------+----------------------+-------------+


In [26]:
# Maximum enrollments per department using aggregation
pipeline = [
    {'$group': {'_id': '$department', 'max_enrollment': {'$max': '$enrollments'}}}
]

courses_data = courses_collection.aggregate(pipeline)

# Show data with table
table = PrettyTable()
table.field_names = ["Department", "Max Enrollment"]

for result in courses_data:
    table.add_row([result['_id'], result['max_enrollment']])

print(table)

+------------------------------------+----------------+
|             Department             | Max Enrollment |
+------------------------------------+----------------+
|            Mathematics             |       55       |
|        Engineering Physics         |       30       |
|          Computer Science          |       60       |
|        Robotics Engineering        |       38       |
|       Physics and Astronomy        |       25       |
|          Applied Physics           |       35       |
| Electrical and Physics Engineering |       28       |
|        Computer Engineering        |       50       |
|              Physics               |       40       |
+------------------------------------+----------------+


In [27]:
# Adding enrollment category field based on enrollments
pipeline = [
    {'$addFields': {'enrollment_category': {'$cond': {'if': {'$gt': ['$enrollments', 20]}, 'then': 'high', 'else': 'low'}}}}
]

courses_data = courses_collection.aggregate(pipeline)

# Show data with table
table = PrettyTable()
table.field_names = ["Course", "Enrollment","Department", "Enrollment Category"]

for result in courses_data:
    table.add_row([result['course'], result['enrollments'], result['department'], result['enrollment_category']])

print(table)

+--------------------------+------------+------------------------------------+---------------------+
|          Course          | Enrollment |             Department             | Enrollment Category |
+--------------------------+------------+------------------------------------+---------------------+
|      Mechanics 101       |     40     |              Physics               |         high        |
|    Quantum Mechanics     |     35     |          Applied Physics           |         high        |
|      Thermodynamics      |     30     |        Engineering Physics         |         high        |
|          Optics          |     25     |       Physics and Astronomy        |         high        |
|     Data Structures      |     60     |          Computer Science          |         high        |
|     Machine Learning     |     50     |        Computer Engineering        |         high        |
|        Calculus I        |     55     |            Mathematics             |         high

In [28]:
# Adding enrollment with 3 category field based on enrollments
pipeline = [
    {'$addFields': {'enrollment_category': {'$switch': {'branches': [
                        {'case': {'$lte': ['$enrollments', 20]}, 'then': 'low'},
                        {'case': {'$and': [{'$gt': ['$enrollments', 20]}, {'$lte': ['$enrollments', 40]}]}, 'then': 'medium'},
                        {'case': {'$gt': ['$enrollments', 40]}, 'then': 'high'}
                    ],'default': 'unknown'}}}}
]
# Collect data from query above
courses_data = courses_collection.aggregate(pipeline)

# Show data with table
table = PrettyTable()
table.field_names = ["Course", "Enrollments", "Department", "Enrollment Category"]

for result in courses_data:
    table.add_row([result['course'], result['enrollments'], result['department'], result['enrollment_category']])

print(table)

+--------------------------+-------------+------------------------------------+---------------------+
|          Course          | Enrollments |             Department             | Enrollment Category |
+--------------------------+-------------+------------------------------------+---------------------+
|      Mechanics 101       |      40     |              Physics               |        medium       |
|    Quantum Mechanics     |      35     |          Applied Physics           |        medium       |
|      Thermodynamics      |      30     |        Engineering Physics         |        medium       |
|          Optics          |      25     |       Physics and Astronomy        |        medium       |
|     Data Structures      |      60     |          Computer Science          |         high        |
|     Machine Learning     |      50     |        Computer Engineering        |         high        |
|        Calculus I        |      55     |            Mathematics             |   

In [22]:
courses_collection.delete_many({})

DeleteResult({'n': 20, 'ok': 1.0}, acknowledged=True)